# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902016


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:04<01:07,  2.32s/it]

Rendering models:  10%|▉         | 3/31 [00:06<00:59,  2.12s/it]

Rendering models:  13%|█▎        | 4/31 [00:06<00:44,  1.63s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:47,  1.81s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:18<00:02,  1.53it/s]

Rendering models:  90%|█████████ | 28/31 [00:19<00:01,  1.71it/s]

Rendering models:  94%|█████████▎| 29/31 [00:19<00:01,  1.67it/s]

Rendering models: 100%|██████████| 31/31 [00:20<00:00,  2.02it/s]

equidad1                              0.001474
loki99                                0.030092
WhittleMario                          0.000628
Flebertabdly                          0.007164
not-logged-in-cae133427d99aac1e554    0.002102
paulogmcruz                           0.001280
not-logged-in-a519260ebf4b014e9045    0.000299
not-logged-in-a87793e85210b5433279    0.000331
kayleebug2017                         0.000688
Thedunkmasta                          0.002352
not-logged-in-f46775ec80a9b6e163ce    0.000387
not-logged-in-dd1f221ba629630c84ec    0.000883
danielrwik                            0.000479
Theaker                               0.002650
nora.zoo                              0.000285
not-logged-in-314d48142a3a122ba666    0.001395
lstallard                             0.001167
not-logged-in-9f30f411df9d28d2c541    0.000552
not-logged-in-a1e053622afe0b0938d8    0.000570
Lavadude                              0.012634
ShirleyL                              0.000337
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())